In [5]:
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()

'C:\\spark-3.1.2-bin-hadoop3.2'

In [6]:
import math
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.functions import lit
from pyspark.sql.types import StringType
from pyspark.sql import SQLContext

In [7]:
spark = SparkSession.builder.appName('SparkSQL_Python_Profeco').getOrCreate()

In [20]:
data_path = 'C:/Users/Jodre/OneDrive/Desktop/Wizeline_Data/'

In [21]:
file_path = data_path + 'all_data.csv'
df = spark.read.format('csv').option("header", "true").load(file_path)

In [22]:
df.show(10)

+--------------------+--------------------+------------------+----------------+----------------+------+--------------------+------------------+--------------------+--------------------+--------------------+----------------+--------------------+--------+----------+
|            producto|        presentacion|             marca|       categoria|        catalogo|precio|       fechaRegistro|   cadenaComercial|                giro|     nombreComercial|           direccion|          estado|           municipio| latitud|  longitud|
+--------------------+--------------------+------------------+----------------+----------------+------+--------------------+------------------+--------------------+--------------------+--------------------+----------------+--------------------+--------+----------+
|CUADERNO FORMA IT...|96 HOJAS PASTA DU...|          ESTRELLA|MATERIAL ESCOLAR|UTILES ESCOLARES|  25.9|2011-05-18 00:00:...|ABASTECEDORA LUMEN|          PAPELERIAS|ABASTECEDORA LUME...|CANNES No. 6 ESQ....

In [24]:
df.columns

['producto',
 'presentacion',
 'marca',
 'categoria',
 'catalogo',
 'precio',
 'fechaRegistro',
 'cadenaComercial',
 'giro',
 'nombreComercial',
 'direccion',
 'estado',
 'municipio',
 'latitud',
 'longitud']

In [31]:
df = df.drop_duplicates()

In [34]:
df = df.na.drop()

In [35]:
df.createOrReplaceTempView('profeco')

In [36]:
spark.sql("Select distinct cadenaComercial " 
                    "from profeco").count()

705

In [37]:
df_sql = spark.sql("select "
    "z.estado, "
    "z.producto, "
    "z.cantidad "
"from ( "
 " select "
 " estado, "
 " max(cantidad) as top_cantidad "
 "  from "
 "  ( "
 "  select "
 "  estado, "
                    "producto, "
                    "count(*) as cantidad "
                "from "
                    "profeco "
                "group by "
                    "estado, "
                    "producto " 
            ") x "
        "group by "
            " estado "
    ") y "
    "join "
        "( "
            "select " 
                "estado, "
                "producto, "
                "count(*) as cantidad "
            "from "
                "profeco "
            "group by "
                "estado, "
                "producto "
        ") z "
    "on "  
        "y.estado = z.estado and "
        "y.top_cantidad = z.cantidad")

In [38]:
df_sql.show(100)

+--------------------+--------------------+--------+
|              estado|            producto|cantidad|
+--------------------+--------------------+--------+
| BAJA CALIFORNIA SUR|            REFRESCO|   26290|
|            GUERRERO|            REFRESCO|    8328|
|              OAXACA|LECHE ULTRAPASTEU...|   17575|
|              MÉXICO|            REFRESCO|  187179|
|             CHIAPAS|            REFRESCO|   13776|
|           QUERÉTARO|LECHE ULTRAPASTEU...|   27050|
|             MORELOS|            REFRESCO|    8939|
|             HIDALGO|            REFRESCO|   17473|
|VERACRUZ DE IGNAC...|            REFRESCO|   13674|
| COL. EDUARDO GUERRA|            REFRESCO|     237|
|             JALISCO|            REFRESCO|   79678|
|      AGUASCALIENTES|                 FUD|   11218|
|           CHIHUAHUA|   DETERGENTE P/ROPA|   14463|
|             NAYARIT|            REFRESCO|    7409|
|          TAMAULIPAS|            REFRESCO|   20867|
|             TABASCO|            REFRESCO|   

In [39]:
df_sql = spark.sql("select cadenaComercial, count(DISTINCT producto) as variedad "
           " from profeco "           
           "group by cadenaComercial "
           "order by variedad desc "
           "LIMIT 1 ")
df_sql.show()

+---------------+--------+
|cadenaComercial|variedad|
+---------------+--------+
|        SORIANA|    1046|
+---------------+--------+

